# AdventureWorks Försäljningsanalys

## Syfte
Analysen undersöker försäljningsdata från AdventureWorks-databasen för att:
- Få en överblick och förstå sortimentet och hur det är fördelat över olika kategorier
- Se trender över tid
- Hitta vilka produkter som säljer bäst och vart
- Presentera affärsstrategiska rekommentationer baserat på datan 

## Datakällor
- **Production**: ProductCategory, ProductSubcategory, Product
- **Sales**: SalesOrderHeader, SalesOrderDetail, Customer, SalesTerritory, Store

---
## Setup
### Imports och databasanslutning

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import urllib

# Visualiseringsinställningar
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# UTF-8 stöd för svenska tecken (å, ä, ö)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# Färgpaletter
COLORS = ['#2ecc71', '#3498db', '#9b59b6', '#e74c3c', '#f39c12', '#1abc9c', '#34495e']
sns.set_palette(COLORS)

In [ ]:
# ===== SQL Server Authentication =====

server = 'localhost\\SQLEXPRESS'
database = 'AdventureWorks2025'
username = 'sa'
password = 'Tenta2024!'
driver = 'ODBC Driver 17 for SQL Server'
connection_string = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes;charset=utf8'
connection_url = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}"

# Skapa engine
engine = create_engine(connection_url)

# Testa anslutningen
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1"))
        print("Anslutning lyckades!")
except Exception as e:
    print(f"Anslutningsfel: {e}")

---
## Visualisering 1: Antal produkter per kategori


**Hur många produkter finns i varje kategori?**

### Metod
Genom att slå samman tre tabeller (ProductCategory -> ProductSubcategory -> Product) går det att räkna antalet unika produkter per huvudkategori, för bäst effektivitet görs aggregeringen i SQL.

In [ ]:
# SQL Query - Antal produkter per kategori
query_1 = """
SELECT 
    pc.Name AS Kategori,
    COUNT(DISTINCT p.ProductID) AS AntalProdukter
FROM Production.ProductCategory pc
INNER JOIN Production.ProductSubcategory psc ON pc.ProductCategoryID = psc.ProductCategoryID
INNER JOIN Production.Product p ON psc.ProductSubcategoryID = p.ProductSubcategoryID
GROUP BY pc.Name
ORDER BY AntalProdukter DESC
"""

df_produkter_kategori = pd.read_sql(query_1, engine)
print("Data hamtad:")
df_produkter_kategori

In [ ]:
# Visualisering 1: Vertikalt stapeldiagram
fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(df_produkter_kategori['Kategori'], 
              df_produkter_kategori['AntalProdukter'],
              color=COLORS[:len(df_produkter_kategori)],
              edgecolor='black',
              linewidth=0.8)

# Visa värden ovanför staplarna
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{int(height)}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom',
                fontweight='bold')

ax.set_title('Antal produkter per kategori', fontsize=16, fontweight='bold')
ax.set_xlabel('Kategori')
ax.set_ylabel('Antal produkter')
ax.set_ylim(0, df_produkter_kategori['AntalProdukter'].max() * 1.15)

plt.tight_layout()
plt.show()

### Reflektion efter första visualiseringen:

Core business- eller smör och bröd, är Components och det är också i den kategorin det finns flest produkter (134 st). 

Bikes och Clothing ligger i mellanskiktet (97 st, 35 st) medan Accessories har minst antal sku:s (29 st).

Detta är tydligt när man tittar på diagrammet. 

Slutsats- Även om Components har flest produkter betyde inte det att den är mest lönsam....




---
## Visualisering 2: Omsättning per kategori


**Vilka kategorier har bäst revenue?**

### Metod
Här har jag slagit samman produkttabeller med SalesOrderDetail, det ger en summering på försäljningen per kategori (sorteras från högst till lägst).

In [ ]:
# SQL Query - Försäljning per produktkategori
query_2 = """
SELECT 
    pc.Name AS Kategori,
    SUM(sod.LineTotal) AS TotalForsaljning
FROM Production.ProductCategory pc
INNER JOIN Production.ProductSubcategory psc ON pc.ProductCategoryID = psc.ProductCategoryID
INNER JOIN Production.Product p ON psc.ProductSubcategoryID = p.ProductSubcategoryID
INNER JOIN Sales.SalesOrderDetail sod ON p.ProductID = sod.ProductID
GROUP BY pc.Name
ORDER BY TotalForsaljning DESC
"""

df_forsaljning_kategori = pd.read_sql(query_2, engine)
print("Data hamtad:")
df_forsaljning_kategori['TotalForsaljning_Formatted'] = df_forsaljning_kategori['TotalForsaljning'].apply(lambda x: f"${x:,.0f}")
df_forsaljning_kategori

In [ ]:
# Visualisering 2: Horisontellt stapeldiagram (högsta överst)
fig, ax = plt.subplots(figsize=(12, 6))

# Sortera för att få högsta överst i horisontellt diagram
df_sorted = df_forsaljning_kategori.sort_values('TotalForsaljning', ascending=True)

bars = ax.barh(df_sorted['Kategori'], 
               df_sorted['TotalForsaljning'] / 1e6,  # Visa i miljoner
               color=COLORS[:len(df_sorted)][::-1],
               edgecolor='black',
               linewidth=0.8)

# Lägg till värden vid staplarna
for bar in bars:
    width = bar.get_width()
    ax.annotate(f'${width:.1f}M',
                xy=(width, bar.get_y() + bar.get_height() / 2),
                xytext=(5, 0),
                textcoords="offset points",
                ha='left', va='center',
                fontweight='bold')

ax.set_title('Total försäljning per produktkategori', fontsize=16, fontweight='bold')
ax.set_xlabel('Försäljning (Miljoner USD)')
ax.set_ylabel('Kategori')
ax.set_xlim(0, df_sorted['TotalForsaljning'].max() / 1e6 * 1.2)

plt.tight_layout()
plt.show()

### Refklektion efter andra visualiseringen.

Högst revenue står Bikes för (94,7MD), vilket i sig inte är märkligt eftersom en cykel kostar mer att köpa.

Clothing är ingen kioskvältare (2,1MD) men skapar merförsäljning, om än inte särskilt mycket. 

Marknadsföringsmässigt är huvudprodukten cyklarna, men components är kategorin som omsätter näst mest även om det bara är 12,4% av det Bikes omsätter. 

Förmodligen är Components kategorin där det finns stört potential till tillväxt. 

---
## Visualisering 3: Försäljningstrender över tid


Hur har försäljningen utvecklats, toppar och dalar?
Här har jag valt att presentera omsättning per månad från SalesOrderHeader.



In [ ]:
# SQL Query - Försäljningstrend per månad
query_3 = """
SELECT 
    YEAR(OrderDate) AS Ar,
    MONTH(OrderDate) AS Manad,
    FORMAT(OrderDate, 'yyyy-MM') AS ArManad,
    SUM(TotalDue) AS TotalForsaljning,
    COUNT(SalesOrderID) AS AntalOrdrar
FROM Sales.SalesOrderHeader
GROUP BY YEAR(OrderDate), MONTH(OrderDate), FORMAT(OrderDate, 'yyyy-MM')
ORDER BY Ar, Manad
"""

df_trend = pd.read_sql(query_3, engine)
print(f"Antal månader: {len(df_trend)}")
df_trend.head(10)

In [ ]:
# Visualisering 3: Linjediagram
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(range(len(df_trend)), 
        df_trend['TotalForsaljning'] / 1e6,
        marker='o',
        markersize=4,
        linewidth=2,
        color='#3498db')

# Markera högsta och lägsta punkterna
max_idx = df_trend['TotalForsaljning'].idxmax()
min_idx = df_trend['TotalForsaljning'].idxmin()

ax.scatter(max_idx, df_trend.loc[max_idx, 'TotalForsaljning'] / 1e6, 
           color='#2ecc71', s=150, zorder=5, label=f"Högst: {df_trend.loc[max_idx, 'ArManad']}")
ax.scatter(min_idx, df_trend.loc[min_idx, 'TotalForsaljning'] / 1e6, 
           color='#e74c3c', s=150, zorder=5, label=f"Lägst: {df_trend.loc[min_idx, 'ArManad']}")

# X-axel labels (visar kvartalsvis för läsbarhet)
tick_positions = range(0, len(df_trend), 3)
tick_labels = [df_trend.loc[i, 'ArManad'] for i in tick_positions]
ax.set_xticks(tick_positions)
ax.set_xticklabels(tick_labels, rotation=45, ha='right')

ax.set_title('Försäljningstrend per månad', fontsize=16, fontweight='bold')
ax.set_xlabel('Tid')
ax.set_ylabel('Försäljning (Miljoner USD)')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Reflektioner efter tredje visualiseringen.

Grafen visar att företaget har mått bra men att något har hänt då försäljningen störtdök abrupt.  

Månaden med högst omsättning illustreras i grönt och månaden med sämst i rött- och dippen har skett på bara en månad.  

Att visualisera med ett linjediagram gör det visuellt tydligt och enklare att se trender över tid och att hitta säsongsmönster. 

Det gör det också enklare att jämföra omsättningen mellan olika kvartal etc.

Här bör bolaget, vilket de redan borde påbörjat, göra en grundlig utredning kring orsaker till varför omsättningen backat så pass brutalt. 

---
## Visualisering 4: Omsättning och antal ordrar på årsbasis.

Här presenteras total omsättning och antal ordrar per år- finns det samband mellan volym och värde?


In [ ]:
# SQL Query - Omsättning och ordrar per år
query_4 = """
SELECT 
    YEAR(OrderDate) AS Ar,
    SUM(TotalDue) AS TotalForsaljning,
    COUNT(SalesOrderID) AS AntalOrdrar
FROM Sales.SalesOrderHeader
GROUP BY YEAR(OrderDate)
ORDER BY Ar
"""

df_ar = pd.read_sql(query_4, engine)
print("Data per ar:")
df_ar['TotalForsaljning_Formatted'] = df_ar['TotalForsaljning'].apply(lambda x: f"${x:,.0f}")
df_ar

In [ ]:
# Visualisering 4: Grupperat stapeldiagram med två y-axlar
fig, ax1 = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_ar))
width = 0.35

# Försäljning (vanster y-axel)
bars1 = ax1.bar(x - width/2, df_ar['TotalForsaljning'] / 1e6, width, 
                label='Försäljning (M USD)', color='#3498db', edgecolor='black')
ax1.set_ylabel('Försäljning (Miljoner USD)', color='#3498db')
ax1.tick_params(axis='y', labelcolor='#3498db')

# Antal ordrar (höger y-axel)
ax2 = ax1.twinx()
bars2 = ax2.bar(x + width/2, df_ar['AntalOrdrar'], width,
                label='Antal ordrar', color='#2ecc71', edgecolor='black')
ax2.set_ylabel('Antal ordrar', color='#2ecc71')
ax2.tick_params(axis='y', labelcolor='#2ecc71')

# X-axel
ax1.set_xticks(x)
ax1.set_xticklabels(df_ar['Ar'].astype(str))
ax1.set_xlabel('Ar')

# Titel
ax1.set_title('Försäljning och antal ordrar per år', fontsize=16, fontweight='bold')
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

# Lägg till värden ovanför staplarna
for bar in bars1:
    height = bar.get_height()
    ax1.annotate(f'${height:.1f}M',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3), textcoords="offset points",
                 ha='center', va='bottom', fontsize=9, color='#3498db')

for bar in bars2:
    height = bar.get_height()
    ax2.annotate(f'{int(height):,}',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3), textcoords="offset points",
                 ha='center', va='bottom', fontsize=9, color='#2ecc71')

plt.tight_layout()
plt.show()

### Reflektioner efter fjärde visualiseringen

Av diagrammet går att utläsa att antalet ordrar har ökat markant mellan 2022 och 2024. 2025 har antalet ordrar minskat mot föregående år, liksom omsättningen. 

Omsättningen har minskat med över 50% vilket är oroande, även om den procentuella minskningen över antalet ordrar inte är lika markant. .
 



---
## Visualisering 5: Top 10 produkter

Vilka top 10 har bolaget- här kombinerade jag Product med SalesOrderDetail och summerar LineTotal per produkt. 



In [ ]:
# SQL Query - Top 10 produkter
query_5 = """
SELECT TOP 10
    p.Name AS Produkt,
    pc.Name AS Kategori,
    SUM(sod.LineTotal) AS TotalForsaljning
FROM Production.Product p
INNER JOIN Sales.SalesOrderDetail sod ON p.ProductID = sod.ProductID
INNER JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
INNER JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
GROUP BY p.Name, pc.Name
ORDER BY TotalForsaljning DESC
"""

df_top10 = pd.read_sql(query_5, engine)
print("Top 10 produkter:")
df_top10['TotalForsaljning_Formatted'] = df_top10['TotalForsaljning'].apply(lambda x: f"${x:,.0f}")
df_top10

In [ ]:
# Visualisering 5: Horisontellt stapeldiagram
fig, ax = plt.subplots(figsize=(12, 8))

# Sortera for att få högsta överst
df_sorted = df_top10.sort_values('TotalForsaljning', ascending=True)

# Färg baserat på kategori
kategori_farger = {'Bikes': '#3498db', 'Components': '#2ecc71', 'Clothing': '#e74c3c', 'Accessories': '#f39c12'}
colors = [kategori_farger.get(k, '#95a5a6') for k in df_sorted['Kategori']]

bars = ax.barh(df_sorted['Produkt'], 
               df_sorted['TotalForsaljning'] / 1e6,
               color=colors,
               edgecolor='black',
               linewidth=0.8)

# Lägg till värden
for bar in bars:
    width = bar.get_width()
    ax.annotate(f'${width:.2f}M',
                xy=(width, bar.get_y() + bar.get_height() / 2),
                xytext=(5, 0),
                textcoords="offset points",
                ha='left', va='center',
                fontweight='bold')

# Kategorier
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color, edgecolor='black', label=kategori) 
                   for kategori, color in kategori_farger.items() if kategori in df_top10['Kategori'].values]
ax.legend(handles=legend_elements, loc='lower right', title='Kategori')

ax.set_title('Top 10 produkter efter försäljning', fontsize=16, fontweight='bold')
ax.set_xlabel('Försäljning (Miljoner USD)')
ax.set_ylabel('Produkt')
ax.set_xlim(0, df_sorted['TotalForsaljning'].max() / 1e6 * 1.2)

plt.tight_layout()
plt.show()

### Femte visualiseringen- Reflektioner

Den bäst säljande produkten ligger högst upp, med omsättningsvärde. 

Som jag tidigare konstaterat är det Bikes som står för högst omsättning, vilket blir glasklart så samtliga produkter på toplistan består av cyklar. 



---
## Visualisering 6: Omsättning och kunder per region

Skiljer sig omsättningen baserat på regioner och hur många är de unika kunderna i varje region? 

Här har jag plockat SalesTerritory och SalesOrderHeader samt Customer för att få både omsättning och unika kunder per region. 



In [ ]:
# SQL Query - Försäljning och kunder per region
query_6 = """
SELECT 
    st.Name AS Region,
    st.[Group] AS RegionGrupp,
    SUM(soh.TotalDue) AS TotalForsaljning,
    COUNT(DISTINCT soh.CustomerID) AS AntalKunder
FROM Sales.SalesTerritory st
INNER JOIN Sales.SalesOrderHeader soh ON st.TerritoryID = soh.TerritoryID
GROUP BY st.Name, st.[Group]
ORDER BY TotalForsaljning DESC
"""

df_region = pd.read_sql(query_6, engine)
print("Försäljning per region:")
df_region['TotalForsaljning_Formatted'] = df_region['TotalForsaljning'].apply(lambda x: f"${x:,.0f}")
df_region

In [ ]:
# Visualisering 6: Grupperat stapeldiagram
fig, ax1 = plt.subplots(figsize=(14, 7))

x = np.arange(len(df_region))
width = 0.35

# Försäljning
bars1 = ax1.bar(x - width/2, df_region['TotalForsaljning'] / 1e6, width,
                label='Försäljning (M USD)', color='#3498db', edgecolor='black')
ax1.set_ylabel('Försäljning (Miljoner USD)', color='#3498db')
ax1.tick_params(axis='y', labelcolor='#3498db')

# Antal kunder (hoger axel)
ax2 = ax1.twinx()
bars2 = ax2.bar(x + width/2, df_region['AntalKunder'], width,
                label='Antal kunder', color='#e74c3c', edgecolor='black')
ax2.set_ylabel('Antal unika kunder', color='#e74c3c')
ax2.tick_params(axis='y', labelcolor='#e74c3c')

ax1.set_xticks(x)
ax1.set_xticklabels(df_region['Region'], rotation=45, ha='right')
ax1.set_xlabel('Region')

ax1.set_title('Försäljning och antal kunder per region', fontsize=16, fontweight='bold')


lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
# Beräkna genomsnittligt ordervärde per kund per region
df_region['SnittPerKund'] = df_region['TotalForsaljning'] / df_region['AntalKunder']
print("\nGenomsnittlig försäljning per kund:")
df_region[['Region', 'TotalForsaljning_Formatted', 'AntalKunder', 'SnittPerKund']].sort_values('SnittPerKund', ascending=False)

### Reflektioner efter visualisering 6

Regionen med högst omsättning är Southwest och det är också den regionen som har flest kunder. 
Central, Southeast och Northeast har få kunder men bra försäljning, här går att spekulera i om det är i de regionerna som det finns fler B2B än privatkunder. 

Här finns massor av WhiteSpace att identifiera och med en sådan analys på plats, en potential för stor tillväxt för bolaget. 



---
## Visualisering 7: Snittorder per region och kundtyp

Hur skiljer sig omsättningen mellan privatkunder och företagskunder och per region?


In [ ]:
# SQL Query - Genomsnittligt ordervärde per region och kundtyp
query_7 = """
SELECT 
    st.Name AS Region,
    CASE WHEN c.StoreID IS NOT NULL THEN 'Store' ELSE 'Individual' END AS Kundtyp,
    SUM(soh.TotalDue) AS TotalForsaljning,
    COUNT(soh.SalesOrderID) AS AntalOrdrar,
    SUM(soh.TotalDue) / COUNT(soh.SalesOrderID) AS SnittOrdenvarde
FROM Sales.SalesTerritory st
INNER JOIN Sales.SalesOrderHeader soh ON st.TerritoryID = soh.TerritoryID
INNER JOIN Sales.Customer c ON soh.CustomerID = c.CustomerID
GROUP BY st.Name, CASE WHEN c.StoreID IS NOT NULL THEN 'Store' ELSE 'Individual' END
ORDER BY Region, Kundtyp
"""

df_kundtyp = pd.read_sql(query_7, engine)
print("Genomsnittligt ordervärde per region och kundtyp:")
df_kundtyp['SnittOrdenvarde_Formatted'] = df_kundtyp['SnittOrdenvarde'].apply(lambda x: f"${x:,.0f}")
df_kundtyp

In [ ]:
# Pivot för visualisering
df_pivot = df_kundtyp.pivot(index='Region', columns='Kundtyp', values='SnittOrdenvarde').fillna(0)

# Beräkna totalsnitt för sortering
df_pivot['Total'] = df_pivot.sum(axis=1)
df_pivot = df_pivot.sort_values('Total', ascending=True)
df_pivot = df_pivot.drop('Total', axis=1)

print("Pivot tabell:")
df_pivot

In [ ]:
# Visualisering 7: Grupperat horisontellt stapeldiagram
fig, ax = plt.subplots(figsize=(12, 8))

y = np.arange(len(df_pivot))
height = 0.35

# Företagskunder/store och privatkunder
if 'Store' in df_pivot.columns:
    bars1 = ax.barh(y - height/2, df_pivot['Store'], height,
                    label='Store (Foretag)', color='#3498db', edgecolor='black')
if 'Individual' in df_pivot.columns:
    bars2 = ax.barh(y + height/2, df_pivot['Individual'], height,
                    label='Individual (Privat)', color='#2ecc71', edgecolor='black')

ax.set_yticks(y)
ax.set_yticklabels(df_pivot.index)
ax.set_xlabel('Genomsnittligt ordervärde (USD)')
ax.set_ylabel('Region')
ax.set_title('Genomsnittligt ordervärde per region och kundtyp', fontsize=16, fontweight='bold')
ax.legend(loc='lower right')

# Formatera x-axeln
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

plt.tight_layout()
plt.show()

### Reflektion - Visualisering 7

Att B2B har högre omsättning än privatkunder är inte så konstigt.

Även här är det tydligt att det finns WhiteSpace att identifiera och plocka hem högre omsättning. 

---
# Analys Kundvärde

Kan man segmentera kunder baserat på beteende och vad är det som karakteriserar en guldkund? 


---
## Kundanalys 1: Top 20 baserat på inköpsvärde

In [ ]:
# SQL Query - Top 20 kunder
query_kund1 = """
SELECT TOP 20
    c.CustomerID,
    CASE WHEN s.Name IS NOT NULL THEN s.Name 
         ELSE CONCAT('Kund ', c.CustomerID) END AS KundNamn,
    CASE WHEN c.StoreID IS NOT NULL THEN 'Store' ELSE 'Individual' END AS Kundtyp,
    st.Name AS Region,
    COUNT(soh.SalesOrderID) AS AntalOrdrar,
    SUM(soh.TotalDue) AS TotalInkop,
    AVG(soh.TotalDue) AS SnittOrder,
    DATEDIFF(DAY, MAX(soh.OrderDate), GETDATE()) AS DagarSedanSenasteOrder
FROM Sales.Customer c
INNER JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
INNER JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
LEFT JOIN Sales.Store s ON c.StoreID = s.BusinessEntityID
GROUP BY c.CustomerID, s.Name, 
         CASE WHEN c.StoreID IS NOT NULL THEN 'Store' ELSE 'Individual' END,
         st.Name
ORDER BY TotalInkop DESC
"""

df_top_kunder = pd.read_sql(query_kund1, engine)
print("Top 20 kunder:")
df_top_kunder

In [ ]:
# Visualisering: Top 20 kunder
fig, ax = plt.subplots(figsize=(14, 10))

# Sortera för att få högsta överst
df_sorted = df_top_kunder.sort_values('TotalInkop', ascending=True)

# Färg baserat på kundtyp
colors = ['#3498db' if k == 'Store' else '#2ecc71' for k in df_sorted['Kundtyp']]

bars = ax.barh(range(len(df_sorted)), 
               df_sorted['TotalInkop'] / 1000,
               color=colors,
               edgecolor='black')

# Labels
ax.set_yticks(range(len(df_sorted)))
ax.set_yticklabels(df_sorted['KundNamn'], fontsize=9)

# Lägg till värden
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax.annotate(f'${width:.0f}K',
                xy=(width, bar.get_y() + bar.get_height() / 2),
                xytext=(3, 0),
                textcoords="offset points",
                ha='left', va='center', fontsize=8)

from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#3498db', label='Store (Företag)'),
                   Patch(facecolor='#2ecc71', label='Individual (Privat)')]
ax.legend(handles=legend_elements, loc='lower right')

ax.set_title('Top 20 kunder efter totalt inköpsvärde', fontsize=16, fontweight='bold')
ax.set_xlabel('Totalt inköpsvärde (Tusen USD)')

plt.tight_layout()
plt.show()

### Reflektion - Top 20 kunder

Merparten av listan består av B2B-kunder – inköpsvärdet för consumers är så pass lågt i förhållande till B2B att de inte syns på topplistan.

Här är en rödflagg för bolaget: risken att ha så stor andel av total omsättning bundet till ett fåtal B2B-kunder. 
Hur täcker de tappen om någon av dessa försvinner? Hur ser pipeline ut med prospects?

# ---
## Kundanalys 2: RFM-segmentering

# Klassificering av kunder baserat på köpbeteende (Recency, Frequency, Monetary)

In [ ]:
# SQL Query - RFM-data för alla kunder
query_rfm = """
SELECT 
    c.CustomerID,
    CASE WHEN c.StoreID IS NOT NULL THEN 'Store' ELSE 'Individual' END AS Kundtyp,
    DATEDIFF(DAY, MAX(soh.OrderDate), (SELECT MAX(OrderDate) FROM Sales.SalesOrderHeader)) AS Recency,
    COUNT(soh.SalesOrderID) AS Frequency,
    SUM(soh.TotalDue) AS Monetary
FROM Sales.Customer c
INNER JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
GROUP BY c.CustomerID, CASE WHEN c.StoreID IS NOT NULL THEN 'Store' ELSE 'Individual' END
"""

df_rfm = pd.read_sql(query_rfm, engine)
print(f"Antal kunder: {len(df_rfm)}")
df_rfm.describe()

In [ ]:
# Skapa RFM-score (1-5 för varje dimension)
df_rfm['R_Score'] = pd.qcut(df_rfm['Recency'], q=5, labels=[5, 4, 3, 2, 1])  # Lägre recency = högre score
df_rfm['F_Score'] = pd.qcut(df_rfm['Frequency'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5])
df_rfm['M_Score'] = pd.qcut(df_rfm['Monetary'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5])

# Skapa RFM-segment
df_rfm['RFM_Score'] = df_rfm['R_Score'].astype(str) + df_rfm['F_Score'].astype(str) + df_rfm['M_Score'].astype(str)
df_rfm['RFM_Total'] = df_rfm['R_Score'].astype(int) + df_rfm['F_Score'].astype(int) + df_rfm['M_Score'].astype(int)

# Segmentera kunder
def segment_customer(row):
    r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Loyal Customers'
    elif r >= 4 and f <= 2:
        return 'New Customers'
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'
    elif r <= 2 and f <= 2:
        return 'Lost'
    else:
        return 'Potential'

df_rfm['Segment'] = df_rfm.apply(segment_customer, axis=1)

print("Kundsegment-fördelning:")
df_rfm['Segment'].value_counts()

In [ ]:
# Sammanställ per segment
segment_summary = df_rfm.groupby('Segment').agg({
    'CustomerID': 'count',
    'Monetary': 'sum',
    'Frequency': 'mean'
}).rename(columns={'CustomerID': 'AntalKunder', 'Monetary': 'TotalVarde'})

# Sortera i logisk ordning
segment_order = ['Champions', 'Loyal Customers', 'Potential', 'New Customers', 'At Risk', 'Lost']
segment_summary = segment_summary.reindex([s for s in segment_order if s in segment_summary.index])

# Färger per segment
segment_colors = {
    'Champions': '#2ecc71',
    'Loyal Customers': '#3498db',
    'New Customers': '#f39c12',
    'Potential': '#9b59b6',
    'At Risk': '#e74c3c',
    'Lost': '#95a5a6'
}
colors = [segment_colors[s] for s in segment_summary.index]

# Visualisering: Två diagram sida vid sida
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Diagram 1: Antal kunder per segment
axes[0].barh(segment_summary.index, segment_summary['AntalKunder'], color=colors, edgecolor='black')
axes[0].set_xlabel('Antal kunder')
axes[0].set_title('Antal kunder per segment', fontweight='bold')
axes[0].invert_yaxis()  # Champions överst

for i, v in enumerate(segment_summary['AntalKunder']):
    axes[0].text(v + 10, i, str(v), va='center', fontweight='bold')

# Diagram 2: Totalt värde per segment
axes[1].barh(segment_summary.index, segment_summary['TotalVarde'] / 1e6, color=colors, edgecolor='black')
axes[1].set_xlabel('Totalt värde (Miljoner USD)')
axes[1].set_title('Totalt kundvärde per segment', fontweight='bold')
axes[1].invert_yaxis()

for i, v in enumerate(segment_summary['TotalVarde'] / 1e6):
    axes[1].text(v + 0.3, i, f'${v:.1f}M', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Visa sammanställning
print("\nSegment-sammanställning:")
segment_summary['SnittFrekvens'] = segment_summary['Frequency'].round(1)
segment_summary['TotalVarde_Formatted'] = segment_summary['TotalVarde'].apply(lambda x: f"${x:,.0f}")
segment_summary[['AntalKunder', 'TotalVarde_Formatted', 'SnittFrekvens']]

### Reflektion - RFM-segmentering

Nu syns det tydligt hur kundsegmenten fördelar sig – både i antal och ekonomiskt värde.

- **Champions** är de mest värdefulla kunderna som köper ofta och nyligen
- **At Risk** är kunder med högt historiskt värde som börjat glida iväg – prioritera uppföljning!
- **Lost** har inte handlat på länge och kräver win-back-kampanjer

Fokus bör ligga på att behålla Champions och rädda At Risk innan de blir Lost.

# ---
## Kundanalys 3: Churn-trend

# Vilka kunder har vi förlorat och när slutade de handla?

In [ ]:
# SQL Query - Churn-trend (kunder som slutat köpa, >180 dagar sedan sista order)
query_churn = """
SELECT 
    FORMAT(LastOrderDate, 'yyyy-QQ') AS SistaOrderKvartal,
    COUNT(CustomerID) AS AntalKunder,
    SUM(TotalSpent) AS TotalVarde
FROM (
    SELECT 
        c.CustomerID,
        MAX(soh.OrderDate) AS LastOrderDate,
        SUM(soh.TotalDue) AS TotalSpent
    FROM Sales.Customer c
    INNER JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    GROUP BY c.CustomerID
    HAVING DATEDIFF(DAY, MAX(soh.OrderDate), (SELECT MAX(OrderDate) FROM Sales.SalesOrderHeader)) > 180
) AS ChurnedCustomers
GROUP BY FORMAT(LastOrderDate, 'yyyy-QQ')
ORDER BY SistaOrderKvartal
"""

df_churn = pd.read_sql(query_churn, engine)
print("Churnade kunder per kvartal (>180 dagar sedan senaste order):")
df_churn['TotalVarde_Formatted'] = df_churn['TotalVarde'].apply(lambda x: f"${x:,.0f}")
df_churn

In [ ]:
# Visualisering: Churn-trend
fig, ax1 = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_churn))
width = 0.4

# Antal churnade kunder
bars = ax1.bar(x, df_churn['AntalKunder'], width, 
               color='#e74c3c', edgecolor='black', alpha=0.8, label='Antal churnade')
ax1.set_ylabel('Antal kunder', color='#e74c3c')
ax1.tick_params(axis='y', labelcolor='#e74c3c')

# Förlorat värde (höger axel)
ax2 = ax1.twinx()
ax2.plot(x, df_churn['TotalVarde'] / 1000, 'o-', 
         color='#3498db', linewidth=2.5, markersize=10, label='Förlorat värde (K USD)')
ax2.set_ylabel('Förlorat kundvärde (Tusen USD)', color='#3498db')
ax2.tick_params(axis='y', labelcolor='#3498db')

ax1.set_xticks(x)
ax1.set_xticklabels(df_churn['SistaOrderKvartal'], rotation=45, ha='right')
ax1.set_xlabel('Kvartal för sista order')
ax1.set_title('Churn-trend: När förlorade vi kunder?', fontsize=16, fontweight='bold')

# Lägg till värden på staplarna
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax1.annotate(f'{int(height)}',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3), textcoords="offset points",
                 ha='center', va='bottom', fontsize=9, color='#e74c3c', fontweight='bold')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()

### Reflektion - Churn-analys

Diagrammet visar när bolaget förlorade kunder (de som inte handlat på >180 dagar).

- **Staplarna** visar antalet förlorade kunder per kvartal
- **Linjen** visar det ekonomiska värdet av dessa förlorade kunder

Detta är kritisk information för att:
- Identifiera perioder med ovanligt högt kundbortfall
- Beräkna den faktiska kostnaden av churn
- Prioritera win-back-kampanjer mot de högvärdeskunder som nyligen försvunnit

# ---
## Kundanalys 4: Kundsegment-pivot och jämförelse i värde

In [ ]:
# Pivot tabell: Segment vs Kundtyp
pivot_segment = pd.pivot_table(
    df_rfm,
    values=['Monetary', 'Frequency', 'CustomerID'],
    index='Segment',
    columns='Kundtyp',
    aggfunc={
        'Monetary': 'sum',
        'Frequency': 'mean',
        'CustomerID': 'count'
    }
)

print("Pivot tabell - Segment vs Kundtyp:")
pivot_segment

In [ ]:
# Visualisering: Segment-fördelning per kundtyp
segment_counts = df_rfm.groupby(['Segment', 'Kundtyp']).size().unstack(fill_value=0)

fig, ax = plt.subplots(figsize=(12, 6))

segment_counts.plot(kind='bar', ax=ax, color=['#3498db', '#2ecc71'], edgecolor='black')

ax.set_title('Antal kunder per segment och kundtyp', fontsize=16, fontweight='bold')
ax.set_xlabel('Segment')
ax.set_ylabel('Antal kunder')
ax.legend(title='Kundtyp')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

### Reflektion - Pivot-analys

B2B-kunder hamnar oftare i de högre segmenten (Champions, Loyal) medan privatkunder har spridning över samtliga kategorier.

**At Risk**-segmentet innehåller värdefulla kunder som bolaget bör prioritera med proaktiv uppföljning innan de blir Lost.

# ---
## Kundanalys 5: Livscykel och köpmönster

In [ ]:
# SQL Query - Kundlivscykel
query_livscykel = """
SELECT 
    YEAR(FirstOrder) AS ForstaOrderAr,
    COUNT(CustomerID) AS AntalKunder,
    SUM(TotalOrders) AS TotaltAntalOrdrar,
    SUM(TotalSpent) AS TotalForsaljning,
    AVG(TotalSpent) AS SnittPerKund
FROM (
    SELECT 
        c.CustomerID,
        MIN(soh.OrderDate) AS FirstOrder,
        COUNT(soh.SalesOrderID) AS TotalOrders,
        SUM(soh.TotalDue) AS TotalSpent
    FROM Sales.Customer c
    INNER JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    GROUP BY c.CustomerID
) AS CustomerStats
GROUP BY YEAR(FirstOrder)
ORDER BY ForstaOrderAr
"""

df_livscykel = pd.read_sql(query_livscykel, engine)
print("Kundanskaffning per år:")
df_livscykel

In [ ]:
# Visualisering: Kundanskaffning och värde
fig, ax1 = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_livscykel))
width = 0.35

bars1 = ax1.bar(x - width/2, df_livscykel['AntalKunder'], width,
                label='Nya kunder', color='#2ecc71', edgecolor='black')
ax1.set_ylabel('Antal nya kunder', color='#2ecc71')

ax2 = ax1.twinx()
ax2.plot(x, df_livscykel['SnittPerKund'] / 1000, 'o-', 
         color='#e74c3c', linewidth=2, markersize=8, label='Snitt per kund')
ax2.set_ylabel('Genomsnittligt kundvärde (Tusen USD)', color='#e74c3c')

ax1.set_xticks(x)
ax1.set_xticklabels(df_livscykel['ForstaOrderAr'].astype(str))
ax1.set_xlabel('År för första order')

ax1.set_title('Kundanskaffning och kundvärde per år', fontsize=16, fontweight='bold')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()

### Reflektion - Kundlivscykel

Kunder som funnits längre har naturligt högre livstidsvärde då de haft mer tid att göra inköp.

Nykundstakten varierar mellan åren och nya kunder har lägre snittvärde – det tar tid att bygga upp kundrelationer.

---
# Sammanfattning 

Informationen som framkommit med den här analysen är viktig för Bolagets fortsatta tillväxt. 
Att identifiera whitespace och att så tydligt se att merparten av omsättningen är allokerad till ett fåtal kunder är information som bör prioriteras och lägga upp strategier kring. 

- Hur kan bolaget få befintliga kunder att köpa från fler kategorier?
- Vilka kunder har dom inte (B2B) och varför?
- Hur ser uppföljning och dialog ut med de största kunderna, finns det någon KAM som bearbetar dem? 
- Hur ska bolaget säkerställa i största möjliga mån att pipen innehåller prospects som har ett högt potentiellt värde så att ett eventuellt tapp inte blir lika kännbart? 
- Hur ser försäljningsmålen ut för säljarna- ingår nykundsberabetning? 






In [ ]:
# Stäng databasanslutningen
engine.dispose()
print("Databasanslutning stängd.")